In [0]:
# Inferential Statistics vs Big Data Analytics
## NYC Yellow Taxi Trips (2022–2024)

from pyspark.sql.functions import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import ttest_ind


### Charger le dataset SAMPLE & EDA

### 

In [0]:
sample_path = "/Volumes/workspace/default/filestore/yellowtaxisample1pct_hybrid_stratified.csv"

df_sample = pd.read_csv(sample_path)
df_sample.shape


In [0]:
df_sample.head()

In [0]:
df_sample.isnull().sum()

Analyse des valeurs existantes

In [0]:
cols_focus = [
    "passenger_count",
    "RatecodeID",
    "store_and_fwd_flag",
    "congestion_surcharge",
    "airport_fee",
]

for col in cols_focus:
    print(f"\n* {col}")
    print(df_sample[col].value_counts())

In [0]:
(df_sample["tpep_pickup_datetime"] > df_sample["tpep_dropoff_datetime"]).sum()


### Nettoyage Sample dataset

In [0]:

# 1/ passenger_count : remplir par la médiane
median_passenger = df_sample["passenger_count"].median()

df_sample["passenger_count"] = df_sample["passenger_count"].fillna(median_passenger)

# 2/RatecodeID : remplir par la valeur la plus fréquente (mode
mode_ratecode = df_sample["RatecodeID"].mode()[0]

df_sample["RatecodeID"] = df_sample["RatecodeID"].fillna(mode_ratecode)

# 3/ store_and_fwd_flag : remplir par "N"
df_sample["store_and_fwd_flag"] = df_sample["store_and_fwd_flag"].fillna("N")

# 4/ congestion_surcharge : remplir par 0
df_sample["congestion_surcharge"] = df_sample["congestion_surcharge"].fillna(0)

# 5/ Fusionner Airport_fee dans airport_fee
df_sample["airport_fee"] = (
    df_sample["airport_fee"]
    .fillna(df_sample["Airport_fee"])
    .fillna(0)
)

# Supprimer la colonne en double
df_sample = df_sample.drop(columns=["Airport_fee"])

# Convertir les colonnes datetime en datetime
df_sample["tpep_pickup_datetime"] = pd.to_datetime(df_sample["tpep_pickup_datetime"])
df_sample["tpep_dropoff_datetime"] = pd.to_datetime(df_sample["tpep_dropoff_datetime"])


mask = df_sample["tpep_pickup_datetime"] > df_sample["tpep_dropoff_datetime"]
df_sample.loc[mask, ["tpep_pickup_datetime", "tpep_dropoff_datetime"]] = (
df_sample.loc[mask, ["tpep_dropoff_datetime", "tpep_pickup_datetime"]].values
)



In [ ]:

df_sample.isnull().sum()

## statistiques inférentielles

In [ ]:
# Prix moyen d’une course
fare = df_sample["fare_amount"].dropna()

mean_fare = fare.mean()
std_fare = fare.std(ddof=1)
n = len(fare)

z = stats.norm.ppf(0.975)  # IC 95 %
margin = z * (std_fare / np.sqrt(n))

ci_fare = (mean_fare - margin, mean_fare + margin)

mean_fare, ci_fare

In [ ]:
# Distance moyenne
distance = df_sample["trip_distance"].dropna()

mean_dist = distance.mean()
std_dist = distance.std(ddof=1)
n = len(distance)

margin = z * (std_dist / np.sqrt(n))
ci_dist = (mean_dist - margin, mean_dist + margin)

mean_dist, ci_dist


In [ ]:
# Durée moyenne des courses
df_sample["duration_min"] = (
    df_sample["tpep_dropoff_datetime"] - df_sample["tpep_pickup_datetime"]
).dt.total_seconds() / 60

duration = df_sample["duration_min"].dropna()

mean_dur = duration.mean()
std_dur = duration.std(ddof=1)
n = len(duration)

margin = z * (std_dur / np.sqrt(n))
ci_dur = (mean_dur - margin, mean_dur + margin)

mean_dur, (mean_dur - margin, mean_dur + margin)

In [ ]:
# Proportion des courses avec tip > 0

tip = (df_sample["tip_amount"] > 0)

p_hat = tip.mean()
n = tip.count()

margin = z * np.sqrt((p_hat * (1 - p_hat)) / n)
ci_tip = (p_hat - margin, p_hat + margin)

p_hat, ci_tip


In [ ]:

# Conversion datetime
df_sample["tpep_pickup_datetime"] = pd.to_datetime(df_sample["tpep_pickup_datetime"])

# Variables temporelles
df_sample["hour"] = df_sample["tpep_pickup_datetime"].dt.hour
df_sample["day_of_week"] = df_sample["tpep_pickup_datetime"].dt.day_name()
df_sample["week"] = df_sample["tpep_pickup_datetime"].dt.isocalendar().week

In [ ]:
n = len(df_sample)

hour_counts = df_sample["hour"].value_counts().sort_index()
hour_prop = hour_counts / n


In [ ]:

z = 1.96

hour_ci = pd.DataFrame({
    "proportion": hour_prop,
    "ci_low": hour_prop - z * np.sqrt((hour_prop * (1 - hour_prop)) / n),
    "ci_high": hour_prop + z * np.sqrt((hour_prop * (1 - hour_prop)) / n)
})

hour_ci.head()


In [ ]:
hour_dist = df_sample.groupby("hour").size()

plt.figure(figsize=(10,4))
hour_dist.plot(kind="bar")
plt.title("Distribution des courses par heure")
plt.xlabel("Heure de la journée")
plt.ylabel("Nombre de courses")
plt.xticks(rotation=0)
plt.grid(axis="y", alpha=0.3)
plt.show()

In [ ]:
day_order = [
    "Monday", "Tuesday", "Wednesday",
    "Thursday", "Friday", "Saturday", "Sunday"
]

day_dist = (
    df_sample.groupby("day_of_week")
    .size()
    .reindex(day_order)
)

plt.figure(figsize=(8,4))
day_dist.plot(kind="bar")
plt.title("Distribution des courses par jour de la semaine")
plt.xlabel("Jour")
plt.ylabel("Nombre de courses")
plt.xticks(rotation=30)
plt.grid(axis="y", alpha=0.3)
plt.show()

In [ ]:
week_dist = df_sample.groupby("week").size()

plt.figure(figsize=(10,4))
week_dist.plot()
plt.title("Distribution des courses par semaine")
plt.xlabel("Semaine de l'année")
plt.ylabel("Nombre de courses")
plt.grid(alpha=0.3)
plt.show()


In [ ]:
prix_par_zone_depart = (
    df_sample.groupby("PULocationID")["fare_amount"]
    .mean()
    .sort_values(ascending=False)
)
top_zones = prix_par_zone_depart.head(10)

plt.figure()
plt.bar(top_zones.index.astype(str), top_zones.values)
plt.xlabel("Pickup LocationID")
plt.ylabel("Prix moyen")
plt.title("Top 10 zones (pickup) avec les fares les plus élevés")
plt.show()

In [ ]:
prix_par_zone_depart = (
    df_sample.groupby("DOLocationID")["fare_amount"]
    .mean()
    .sort_values(ascending=False)
)
top_zones = prix_par_zone_depart.head(10)

plt.figure()
plt.bar(top_zones.index.astype(str), top_zones.values)
plt.xlabel("dropoff DOLocationID")
plt.ylabel("Prix moyen")
plt.title("Top 10 zones (dropoff) avec les fares les plus élevés")
plt.show()

In [ ]:
# Distribution géographique (proportions)
zone_dist = df_sample["PULocationID"].value_counts(normalize=True).head(10)

plt.figure(figsize=(10,4))
zone_dist.plot(kind="bar")
plt.title("Top 10 des zones de départ (PULocationID) – Sample")
plt.xlabel("PULocationID")
plt.ylabel("Proportion des courses")
plt.xticks(rotation=45)
plt.grid(axis="y", alpha=0.3)
plt.show()

In [ ]:
# Outliers
# filtrer les courses > 200
df_expensive = df_sample[df_sample["fare_amount"] > 200]
data = df_expensive["fare_amount"].dropna()

mean = data.mean()
std = data.std()
n = len(data)

z = 1.96  # IC à 95%
margin = z * (std / np.sqrt(n))

ic_low = mean - margin
ic_high = mean + margin

mean, ic_low, ic_high

In [ ]:

# filtrer les longues courses
df_long = df_sample[df_sample["trip_distance"] > 30]

data = df_long["trip_distance"].dropna()

mean = data.mean()
std = data.std()
n = len(data)

z = 1.96
margin = z * (std / np.sqrt(n))

ic_low = mean - margin
ic_high = mean + margin

mean, ic_low, ic_high

In [ ]:
# Tip / Fare

df_sample["tip_ratio"] = df_sample["tip_amount"] / df_sample["fare_amount"]

tip_ratio_mean = df_sample["tip_ratio"].dropna().mean()
tip_ratio_mean

In [ ]:
# Ratio moyen tip/fare par type de paiement

df_pay = df_sample[
    (df_sample["payment_type"].isin([1, 2])) &
    (df_sample["fare_amount"] > 0)
].copy()

df_pay["tip_ratio"] = df_pay["tip_amount"] / df_pay["fare_amount"]

tip_ratio_by_payment = df_pay.groupby("payment_type")["tip_ratio"].mean()


tip_ratio_by_payment.rename({
    1: "Credit Card",
    2: "Cash"
})


In [ ]:
card_tips = df_pay[df_pay["payment_type"] == 1]["tip_ratio"].dropna()
cash_tips = df_pay[df_pay["payment_type"] == 2]["tip_ratio"].dropna()

t_stat, p_value = ttest_ind(card_tips, cash_tips, equal_var=False)

t_stat, p_value

## Charger le dataset POPULATION

In [ ]:
pop_path = "/Volumes/workspace/default/filestore/Data_as_paquets/"

df_pop = spark.read.parquet(pop_path)

df_pop.printSchema()
df_pop.count()

### **📌 Description des colonnes – NYC Yellow Taxi**
### 
**VendorID**
Identifiant du fournisseur de taxi **(ex : entreprise ou système de dispatch)**.

**tpep_pickup_datetime**
Date et heure de début de la course **(prise en charge du client)**.

**tpep_dropoff_datetime**
Date et heure de fin de la course.

**passenger_count**
Nombre de passagers dans le taxi.

**trip_distance**
Distance de la course en miles.

**RatecodeID**
Code tarifaire appliqué à la course **(tarif standard, aéroport, hors ville, etc.)**.

**store_and_fwd_flag**
Indique si les données de la course ont été stockées temporairement avant transmission
**(Y = oui, N = non)**.

**PULocationID**
Identifiant de la zone de prise en charge **(Pickup Location)**.

**DOLocationID**
Identifiant de la zone de dépose **(Dropoff Location)**.

**payment_type**
Méthode de paiement utilisée :

1 → Carte

2 → Cash

Autres → non standard

**fare_amount**
Montant de base de la course **(hors taxes et suppléments)**.

**extra**
Suppléments **(heures de nuit, heures de pointe, etc.)**.

**mta_tax**
Taxe MTA **(taxe fixe de transport à NYC)**.

**tip_amount**
Montant du pourboire.

**tolls_amount**
Frais de péage.

**improvement_surcharge**
Supplément réglementaire pour amélioration du service.

**congestion_surcharge**
Supplément lié à la congestion dans certaines zones.

**airport_fee**
Frais supplémentaires pour les trajets vers/depuis les aéroports.

**total_amount**
Montant total payé par le client **(tous frais inclus)**.

In [0]:
files = dbutils.fs.ls(pop_path)

df_list = []
for f in files:
    df_list.append(spark.read.parquet(f.path))

df_pop = df_list[0]
for df in df_list[1:]:
    df_pop = df_pop.unionByName(df, allowMissingColumns=True)

df_pop.count()


In [0]:
# Calcul des NULL
null_summary = df_pop.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df_pop.columns
])

# Récupérer les résultats en dictionnaire
null_counts = null_summary.first().asDict()

# Garder uniquement les colonnes avec NULL
cols_with_nulls = [c for c, v in null_counts.items() if v > 0]

### les colonnes avec des NULL

In [0]:
# Afficher le résumé des NULL pour ces colonnes
null_summary.select(cols_with_nulls).show(truncate=False)


In [0]:
# Vérifier les incohérences de dates
from pyspark.sql.functions import col

invalid_count = df_pop.filter(
    col("tpep_pickup_datetime") > col("tpep_dropoff_datetime")
).count()

print(f"Nombre de courses avec date début > date fin : {invalid_count}")



In [0]:
# renommer la colonne Airport_fee en airport_fee
from pyspark.sql.functions import col

df_pop = df_pop.withColumnRenamed("Airport_fee", "airport_fee")


In [0]:
# Vérifier les types
from pyspark.sql.types import IntegerType, DoubleType, LongType

df_pop = df_pop \
    .withColumn("VendorID", col("VendorID").cast(LongType())) \
    .withColumn("passenger_count", col("passenger_count").cast(IntegerType())) \
    .withColumn("RatecodeID", col("RatecodeID").cast(IntegerType())) \
    .withColumn("PULocationID", col("PULocationID").cast(IntegerType())) \
    .withColumn("DOLocationID", col("DOLocationID").cast(IntegerType())) \
    .withColumn("payment_type", col("payment_type").cast(IntegerType()))


In [0]:
# Corriger les incohérences de dates
from pyspark.sql.functions import when

df_pop_clean = (
    df_pop
    .withColumn(
        "pickup_tmp",
        when(
            col("tpep_pickup_datetime") > col("tpep_dropoff_datetime"),
            col("tpep_dropoff_datetime")
        ).otherwise(col("tpep_pickup_datetime"))
    )
    .withColumn(
        "dropoff_tmp",
        when(
            col("tpep_pickup_datetime") > col("tpep_dropoff_datetime"),
            col("tpep_pickup_datetime")
        ).otherwise(col("tpep_dropoff_datetime"))
    )
    .drop("tpep_pickup_datetime", "tpep_dropoff_datetime")
    .withColumnRenamed("pickup_tmp", "tpep_pickup_datetime")
    .withColumnRenamed("dropoff_tmp", "tpep_dropoff_datetime")
)


In [0]:
# Remplir les valeurs manquantes
from pyspark.sql.functions import when

df_pop_clean = df_pop \
    .fillna({"store_and_fwd_flag": "N"}) \
    .fillna({"congestion_surcharge": 0, "airport_fee": 0}) \
    .withColumn(
        "passenger_count",
        when(col("passenger_count").isNull(), 1).otherwise(col("passenger_count"))
    ) \
    .withColumn(
        "RatecodeID",
        when(col("RatecodeID").isNull(), 1).otherwise(col("RatecodeID"))
    )


In [0]:
#filtrer les incohérences de dates
df_pop_clean.filter(
    col("tpep_pickup_datetime") > col("tpep_dropoff_datetime")
).count()


In [0]:
# Vérifier les valeurs manquantes
df_pop_clean.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df_pop_clean.columns
]).show()


In [0]:
# Calculer la durée du trajet
from pyspark.sql.functions import unix_timestamp

df_pop_clean = df_pop_clean.withColumn(
    "trip_duration_min",
    (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")) / 60
).filter(col("trip_duration_min") > 0)


In [0]:
#1. Prix moyen d’une course
from pyspark.sql.functions import avg

df_pop.select(avg("fare_amount").alias("avg_fare_pop")).show()


In [0]:
#2. Distance moyenne
df_pop.select(avg("trip_distance").alias("avg_distance_pop")).show()


In [0]:
#3. Durée moyenne des courses
from pyspark.sql.functions import unix_timestamp

df_pop_duration = df_pop.withColumn(
    "trip_duration_min",
    (unix_timestamp("tpep_dropoff_datetime") - 
     unix_timestamp("tpep_pickup_datetime")) / 60
)
df_pop_duration.select(avg("trip_duration_min").alias("avg_duration_pop")).show()


In [0]:
#4. Proportion des courses avec tip > 0
from pyspark.sql.functions import when, count

tip_pop = df_pop.select(
    (count(when(col("tip_amount") > 0, True)) / count("*"))
    .alias("tip_rate_pop")
)
tip_pop.show()

In [0]:
#5. Heures de pointe
from pyspark.sql.functions import hour

df_pop.groupBy(hour("tpep_pickup_datetime").alias("hour")) \
      .count().orderBy("hour").show()


In [0]:
#Top 10 zones de pickup avec les fares moyens les plus élevés
from pyspark.sql.functions import avg

prix_par_zone_pickup_pop = (
    df_pop
    .groupBy("PULocationID")
    .agg(avg("fare_amount").alias("avg_fare"))
    .orderBy("avg_fare", ascending=False)
)

prix_par_zone_pickup_pop.show(10)

In [0]:
#Top 10 zones de dropoff avec les fares moyens les plus élevés
prix_par_zone_dropoff_pop = (
    df_pop
    .groupBy("DOLocationID")
    .agg(avg("fare_amount").alias("avg_fare"))
    .orderBy("avg_fare", ascending=False)
)

prix_par_zone_dropoff_pop.show(10)

In [0]:
# Distribution par heure
from pyspark.sql.functions import hour

hour_pop = (
    df_pop
    .withColumn("hour", hour("tpep_pickup_datetime"))
    .groupBy("hour")
    .count()
    .withColumn("proportion", col("count") / total_pop)
    .orderBy("hour")
)

hour_pop.show()

In [0]:
#Distribution par JOUR DE LA SEMAINE
from pyspark.sql.functions import date_format

day_pop = (
    df_pop
    .withColumn("day_of_week", date_format("tpep_pickup_datetime", "EEEE"))
    .groupBy("day_of_week")
    .count()
    .withColumn("proportion", col("count") / total_pop)
)

day_pop.show()

In [0]:
from pyspark.sql.functions import weekofyear, col
#Créer la variable semaine de l’année
df_pop_week = df_pop.withColumn(
    "week",
    weekofyear(col("tpep_pickup_datetime"))
)


In [0]:
#Nombre de courses par semaine (population)
eek_dist_pop = (
    df_pop_week
    .groupBy("week")
    .count()
    .orderBy("week")
)
week_dist_pop.show(10)


In [0]:
#Proportion des courses par semaine
week_dist_pop.orderBy(col("proportion").desc()).show()


In [0]:
# Calculer les courses les plus fréquentes par quartier
geo_pop = df_pop_clean.groupBy("PULocationID").count().orderBy("count", ascending=False)
geo_pop.show(10)


In [0]:
df_pop_clean.select(
    "fare_amount",
    "trip_distance",
    "tip_amount",
    "passenger_count",
    "total_amount"
).describe().show()


In [0]:
outliers_pop = df_pop_clean.filter(
    (col("fare_amount") > 200) |
    (col("trip_distance") > 50) |
    (col("trip_duration_min") > 180)
)

outliers_pop.count()


In [0]:
#Courses très chères (fare_amount > 200)
from pyspark.sql.functions import avg, col

# filtrer les courses très chères (population)
df_expensive_pop = df_pop.filter(col("fare_amount") > 200)

# moyenne des fares élevées
df_expensive_pop.select(
    avg("fare_amount").alias("mean_fare_expensive_pop")
).show()

In [0]:
#Courses très longues (trip_distance > 30 miles)
# filtrer les courses très longues (population)
df_long_pop = df_pop.filter(col("trip_distance") > 30)

# moyenne des distances longues
df_long_pop.select(
    avg("trip_distance").alias("mean_distance_long_pop")
).show()

In [0]:
#8. Ratio tip / fare par type de paiement
from pyspark.sql.functions import sum
# Calculer le pourcentage de pourboire par mode de paiement
ratio_pop = df_pop_clean.groupBy("payment_type").agg(
    (sum("tip_amount") / sum("fare_amount")).alias("tip_fare_ratio")
)
ratio_pop.show()
